In [1]:
import os
from openai import OpenAI
import openai
import json
from sqlalchemy.engine import create_engine
from sqlalchemy import text
import pandas as pd
import time
from pandas import DataFrame
import random

In [2]:
"""A utility function to execute prompts against the LLM"""

def llm_model(prompt):
    client = OpenAI(api_key="")
    try:
        response =client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You have an extensive knowledge on english and have an amazing skill to summarise documents without losing important information and delvireing the content in a very consize way"},
                      {"role": "user", "content": prompt}]
        )
        response_message = response.choices[0].message.content
        return response_message
    except openai.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return "Error with OpenAI API"
    except Exception as e:
        print(f"General error: {e}")
        return "General error in classification"

In [3]:
"""A utility function to setup the connection to the realtime source"""

def create_kafka_engine(connection_params: str):
    
    connect_args = {
        "session_properties": {"streambased_connection": connection_params, "use_streambased": True},
        "http_scheme": "https",
        "schema": "streambased"
    }
    
    engine = create_engine("trino://streambased.cloud:8443/kafka", connect_args=connect_args)
    
    return engine

In [4]:
"""Make a connection for the realtime source"""

connection_params = """{
  "bootstrap.servers": "pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092",
  "security.protocol": "SASL_SSL",
  "sasl.jaas.config": "org.apache.kafka.common.security.plain.PlainLoginModule required username='4OOK2MI6TA37CC4K' password='0mzIsqc1Soz2thm9EutDuAkARY8CZqaNzJ5IF/KboX3l58XN/LZ+yQeuG3LmheIc';",
  "sasl.mechanism": "PLAIN",
  "schema.registry.url":"https://psrc-571d82.europe-west2.gcp.confluent.cloud",
  "basic.auth.credentials.source":"USER_INFO",
  "basic.auth.user.info":"FM36ASJ3BX2SWVXV:gKeS/p/CyfQYfd9/9diFwU6LsauvJ4mtmnPAv68ro7Bm81aZp29QOTXkV5yuv8af"
}"""

engine = create_kafka_engine(connection_params)
connection = engine.connect()

In [14]:
"""Prepare 7 days worth of queries"""

current_timestamp = int(time.time()*1000)
day1_timestamp = current_timestamp - 86400000
day2_timestamp = day1_timestamp - 86400000
day3_timestamp = day2_timestamp - 86400000
day4_timestamp = day3_timestamp - 86400000
day5_timestamp = day4_timestamp - 86400000
day6_timestamp = day5_timestamp - 86400000
day7_timestamp = day6_timestamp - 86400000

"""We limit the rows returned for demo performance reasons"""

day1_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000" 
day2_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day3_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day4_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day5_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day6_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"
day7_query = f"SELECT * FROM yt_gaming_subtitles WHERE timestamp > {day1_timestamp} AND timestamp < {current_timestamp} LIMIT 5000"


In [15]:
"""Execute 7 days worth of queries - in production this would be executed asynchronously and continuously"""

connection = engine.connect()

rs1 = connection.execute(text(day1_query))
day1_l0_df = DataFrame(rs1.fetchall())
day1_l0_df.columns = rs1.keys()

In [16]:
rs2 = connection.execute(text(day2_query))
day2_l0_df = DataFrame(rs2.fetchall())
day2_l0_df.columns = rs2.keys()

In [17]:
rs3 = connection.execute(text(day3_query))
day3_l0_df = DataFrame(rs3.fetchall())
day3_l0_df.columns = rs3.keys()

In [18]:
rs4 = connection.execute(text(day4_query))
day4_l0_df = DataFrame(rs4.fetchall())
day4_l0_df.columns = rs4.keys()

In [19]:
rs5 = connection.execute(text(day5_query))
day5_l0_df = DataFrame(rs5.fetchall())
day5_l0_df.columns = rs5.keys()

In [20]:
rs6 = connection.execute(text(day6_query))
day6_l0_df = DataFrame(rs6.fetchall())
day6_l0_df.columns = rs6.keys()

In [21]:
rs7 = connection.execute(text(day7_query))
day7_l0_df = DataFrame(rs7.fetchall())
day7_l0_df.columns = rs7.keys()

In [22]:
"""Level 1 summarisation: we take 1 day's worth of captions and summarise them to 1000 words"""

day2_l1_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please summmarise
it to 1000 words and make sure you don't leave important stuff out. The captions are: {' '.join(day2_l0_df["text"].dropna())}"""
day2_l1_summary = llm_model(day2_l1_prompt)

In [23]:
day3_l1_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please summmarise
it to 1000 words and make sure you don't leave important stuff out. The captions are: {' '.join(day3_l0_df["text"].dropna())}"""
day3_l1_summary = llm_model(day3_l1_prompt)

In [24]:
"""Level 2 summarisation: we take 1 day's worth of captions and extract 100 keywords"""

day4_l2_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please extract 100 keywords from it. The captions are: {' '.join(day4_l0_df["text"].dropna())}"""
day4_l2_summary = llm_model(day4_l2_prompt)

In [25]:
day5_l2_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please extract 100 keywords from it. The captions are: {' '.join(day5_l0_df["text"].dropna())}"""
day5_l2_summary = llm_model(day5_l2_prompt)

In [26]:
day6_l2_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please extract 100 keywords from it. The captions are: {' '.join(day6_l0_df["text"].dropna())}"""
day6_l2_summary = llm_model(day6_l2_prompt)

In [27]:
day7_l2_prompt = f"""I have provided the input of the youtube gaming subtitles for a single day please extract 100 keywords from it. The captions are: {' '.join(day7_l0_df["text"].dropna())}"""
day7_l2_summary = llm_model(day7_l2_prompt)

In [28]:
"""Now we can compose a prompt"""

prompt = f"""I have provided captions from Youtube gaming videos from the last day here: 

{' '.join(day1_l0_df["text"].dropna())}.

I have also provided a sumary of Youtube gaming videos from 24 to 72hrs ago here: 

{day2_l1_summary}
{day3_l1_summary}

I have also provided keywords extracted from Youtube gaming videos older than 72hrs here:

{day4_l2_summary}
{day5_l2_summary}
{day6_l2_summary}
{day7_l2_summary}

Use this information to answer the user's question. The user's question is what are the latest and enduring trends in the gaming field?"""

print(f"Composed a prompt with length: {len(prompt.split())}")


Composed a prompt with length: 36740


In [29]:
"""Execute the prompt against the LLM"""

result = llm_model(prompt)
print(result)

The latest and enduring trends in the gaming field, as reflected in the provided YouTube gaming content, can be summarized as follows:

1. **Community Engagement and Collaboration**: Many gaming videos highlight the importance of teamwork and communication among players. This is evidenced in multiplayer gaming sessions where players strategize together, showcasing that cooperative gameplay is both enjoyable and essential for success.

2. **Competitive Spirit and Challenges**: Competitive gaming is a prominent theme, with players emphasizing strategies to secure victory and discussing defensive tactics. Players often engage in challenges that not only test their skills but also foster a fun and spirited atmosphere.

3. **Content Creator Dynamics**: There is an ongoing discussion around cancel culture among creators, which influences how content is shaped and perceived. Content creators often explore their digital personas and community feedback, reflecting a broader commentary on social

In [30]:
"""Let's compare with a similar prompt size with a lot less detail"""

top_k_selections = [day1_l0_df,day2_l0_df,day3_l0_df,day4_l0_df,day5_l0_df,day6_l0_df,day7_l0_df]
top_k_df_1 = random.choice(top_k_selections)
top_k_df_2 = random.choice(top_k_selections)

bad_prompt = f"""I have provided captions from Youtube gaming videos here: 

{' '.join(top_k_df_1["text"].dropna())}
{' '.join(top_k_df_2["text"].dropna())}

Use this information to answer the user's question. The user's question is what are the latest and enduring trends in the gaming field?"""

print(f"Composed a prompt with length: {len(bad_prompt.split())}")

Composed a prompt with length: 73197


In [31]:
"""Execute the bad prompt against the LLM"""

bad_result = llm_model(bad_prompt)
print(bad_result)

The latest and enduring trends in the gaming field include:

1. **Live Service Games**: Many developers are focusing on creating live service games that offer ongoing content updates, events, and expansions, allowing players to engage continuously. This trend is seen in popular titles that often include seasonal content and player engagement strategies.

2. **Cross-Platform Play**: There's a growing push for cross-platform capabilities where players on different systems (PC, consoles, mobile) can interact and play together, enhancing community engagement and multiplayer experiences.

3. **Streaming and Content Creation**: The rise of platforms like Twitch and YouTube has transformed gaming into a spectator sport, with many gamers becoming content creators. Streaming various games, esports competitions, and gaming-related content has become a major part of gaming culture.

4. **Mobile Gaming**: There’s a significant increase in mobile gaming popularity, with many developers focusing on 

In [32]:
raw_prompt = "what are the latest and enduring trends in the gaming field?"
raw_result = llm_model(raw_prompt)

print(raw_result)

As of late 2023, several key trends are shaping the gaming industry:

1. **Cloud Gaming**: Services like Google Stadia and Xbox Cloud Gaming are making high-quality gaming more accessible, allowing players to stream games without high-end hardware.

2. **Cross-Platform Play**: Increasingly, games are allowing players on different platforms to play together, enhancing social interaction and broadening player bases.

3. **Mobile Gaming Expansion**: The mobile gaming market continues to grow, driven by innovative game design and the rise of casual gaming.

4. **Esports Growth**: Competitive gaming is becoming mainstream, with larger viewership, sponsorships, and professional leagues, attracting significant investment.

5. **Virtual Reality (VR) and Augmented Reality (AR)**: Continued advancements in VR and AR technologies are creating more immersive gaming experiences.

6. **Indie Game Renaissance**: Indie developers are gaining prominence, delivering unique, high-quality games that chall